In [1]:
import pandas as pd 
import re
import numpy as np

In [2]:
teams = [
    'Arsenal', 'Manchester City', 'Tottenham Hotspur', 'Brighton & Hove Albion', 'Manchester United', 'Chelsea', 'Liverpool', 'Brentford', 'Leeds United', 'Fulham', 'Newcastle United',
    'Southampton', 'AFC Bournemouth', 'Wolverhampton Wanderers', 'Crystal Palace', 'Everton', 'Aston Villa', 'West Ham United', 'Nottingham Forest', 'Leicester City'
]

teams_list = ('|'.join(teams))

In [3]:
results = [
    'Arsenal 2 Aston Villa 1', 'AFC Bournemouth 0 Wolverhampton Wanderers 0', 'Manchester City 6 Nottingham Forest 0', 'West Ham United 1 Tottenham Hotspur 1',
    'Liverpool 2 Newcastle United 1', 'Crystal Palace 1 Brentford 1', 'Fulham 2 Brighton & Hove Albion 1', 'Southampton 2 Chelsea 1', 'Leeds United 1 Everton 1',
    'Aston Villa 0 West Ham United 1', 'Wolverhampton Wanderers 1 Newcastle United 1', 'Nottingham Forest 0 Tottenham Hotspur 2', 'Southampton 0 Manchester United 1', 
    'Brentford 1 Everton 1', 'Brighton & Hove Albion 1 Leeds United 0', 'Chelsea 2 Leicester City 1', 'Liverpool 9 AFC Bournemouth 0', 'Manchester City 4 Crystal Palace 2',
    'Arsenal 2 Fulham 1', 'Manchester United 2 Liverpool 1', 'Leeds United 3 Chelsea 0', 'West Ham United 0 Brighton & Hove Albion 2', 'Newcastle United 3 Manchester City 3', 
    'Tottenham Hotspur 1 Wolverhampton Wanderers 0','Crystal Palace 3 Aston Villa 1', 'Everton 1 Nottingham Forest 1', 'Fulham 3 Brentford 2', 'Leicester City 1 Southampton 2', 
    'AFC Bournemouth 0 Arsenal 3', 'Liverpool 1 Crystal Palace 1', 'Nottingham Forest 1 West Ham United 0', 'Chelsea 2 Tottenham Hotspur 2', 'Aston Villa 2 Everton 1', 
    'Arsenal 4 Leicester City 2', 'Brighton & Hove Albion 0 Newcastle United 0', 'Manchester City 4 AFC Bournemouth 0', 'Southampton 2 Leeds United 2', 
    'Wolverhampton Wanderers 0 Fulham 0', 'Brentford 4 Manchester United 0', 'Leicester City 2 Brentford 2', 'Manchester United 1 Brighton & Hove Albion 2',
    'West Ham United 0 Manchester City 2', 'Fulham 2 Liverpool 2', 'AFC Bournemouth 2 Aston Villa 0', 'Leeds United 2 Wolverhampton Wanderers 1', 
    'Newcastle United 2 Nottingham Forest 0', 'Tottenham Hotspur 4 Southampton 1', 'Everton 0 Chelsea 1', 'Crystal Palace 0 Arsenal 2', 'Brighton & Hove Albion 5 Leicester City 2',
    'Manchester United 3 Arsenal 1', 'Everton 0 Liverpool 0', 'Brentford 5 Leeds United 2', 'Chelsea 2 West Ham United 1', 'Newcastle United 0 Crystal Palace 0',
    'Nottingham Forest 2 AFC Bournemouth 3', 'Tottenham Hotspur 2 Fulham 1', 'Wolverhampton Wanderers 1 Southampton 0', 'Aston Villa 1 Manchester City 1', 'Leicester City 0 Manchester United 1'
]

df = pd.DataFrame(results)

In [4]:
df['teams'] = df[0].apply(lambda x: re.findall(teams_list, x))
df['score'] = df[0].apply(lambda x: re.findall("\d", x))

In [5]:
def f(x):
    if x[0] > x[1]:
        return 'home win'
    elif x[1] > x[0]:
        return 'away win'
    else:
        return 'draw'
    
df['result'] = df['score'].apply(lambda x: f(x))

In [6]:
df[['home_team', 'away_team']] = pd.DataFrame(df.teams.tolist()) 
df[['home_goals', 'away_goals']] = pd.DataFrame(df.score.tolist()) 

df[['home_goals', 'away_goals']] = df[['home_goals', 'away_goals']].astype(int)

In [7]:
wins = []
draws = []
loss = []

for result, home_team in zip(df['result'], df['home_team']):
    if result == 'home win':
        wins.append(home_team)
    elif result == 'draw':
        draws.append(home_team) 
    elif result == 'away win':
        loss.append(home_team)    
           

for result, away_team in zip(df['result'], df['away_team']):
    if result == 'away win':
        wins.append(away_team)  
    elif result == 'draw':
        draws.append(away_team)    
    elif result == 'home win':
        loss.append(away_team)         

In [8]:
premier_league = pd.DataFrame(columns=['Teams', 'Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', 'Points'])

In [9]:
premier_league['Teams'] = teams

games_played = df['home_team'].value_counts() + df['away_team'].value_counts()

premier_league['Played'] = premier_league['Teams'].map(games_played.to_dict())

In [10]:
premier_league['Won'] = premier_league['Teams'].apply(lambda x: wins.count(x))
premier_league['Drawn'] = premier_league['Teams'].apply(lambda x: draws.count(x))
premier_league['Lost'] = premier_league['Teams'].apply(lambda x: loss.count(x))

In [11]:
premier_league['GF'] = premier_league['Teams'].apply(lambda x: (df.loc[(df['home_team']==x), 'home_goals'].sum() + df.loc[(df['away_team']==x), 'away_goals'].sum()))
premier_league['GA'] = premier_league['Teams'].apply(lambda x: (df.loc[(df['home_team']==x), 'away_goals'].sum() + df.loc[(df['away_team']==x), 'home_goals'].sum()))

premier_league['GD'] = premier_league['GF'] - premier_league['GA']

In [12]:
premier_league['Points'] = (premier_league['Won'] * 3) + premier_league['Drawn']

In [13]:
premier_league.sort_values(by=['Points','GD'], ascending=False)

,Teams,Played,Won,Drawn,Lost,GF,GA,GD,Points
0,Arsenal,6,5,0,1,14,7,7,15
1,Manchester City,6,4,2,0,20,6,14,14
2,Tottenham Hotspur,6,4,2,0,12,5,7,14
3,Brighton & Hove Albion,6,4,1,1,11,5,6,13
4,Manchester United,6,4,0,2,8,8,0,12
5,Chelsea,6,3,1,2,8,9,-1,10
6,Liverpool,6,2,3,1,15,6,9,9
7,Brentford,6,2,3,1,15,9,6,9
8,Leeds United,6,2,2,2,10,10,0,8
9,Fulham,6,2,2,2,9,9,0,8


In [14]:
data_fixtures = pd.read_csv('prem fixtures.csv')

In [15]:
data_fixtures['Fixture'] = data_fixtures['Fixture'].replace("LiverpoolTBCTo be ConfirmedLeeds United", "Liverpool15:00Leeds United")
data_fixtures['Fixture'] = data_fixtures['Fixture'].replace("ChelseaTBCTo be ConfirmedManchester United", "Chelsea15:00Manchester United")

In [16]:
data_fixtures['Fixture'] = data_fixtures['Fixture'].str.split("\d\d:\d\d")

In [17]:
premier_league['fixtures'] = premier_league['Teams'].apply(lambda x: [y for y in data_fixtures['Fixture'] if x in y])

In [18]:
premier_league['GW7'] = [x[0] for x in premier_league['fixtures']]
premier_league['GW8'] = [x[1] for x in premier_league['fixtures']]
premier_league['GW9'] = [x[2] for x in premier_league['fixtures']]
premier_league['GW10'] = [x[3] for x in premier_league['fixtures']]
premier_league['GW11'] = [x[4] for x in premier_league['fixtures']]
premier_league['GW12'] = [x[5] for x in premier_league['fixtures']]
premier_league['GW13'] = [x[6] for x in premier_league['fixtures']]
premier_league['GW14'] = [x[7] for x in premier_league['fixtures']]
premier_league['GW15'] = [x[8] for x in premier_league['fixtures']]
premier_league['GW16'] = [x[9] for x in premier_league['fixtures']]


In [19]:
def home_away(x):
    result = []
    for team, fixture in zip(premier_league.Teams, x):
        if team == fixture[0]:
            result.append(fixture[1] + " (H)")
        elif team == fixture[1]:
            result.append(fixture[0] + " (A)")
    return result

In [20]:
premier_league['GW7'] = home_away(premier_league.GW7)
premier_league['GW8'] = home_away(premier_league.GW8)
premier_league['GW9'] = home_away(premier_league.GW9)
premier_league['GW10'] = home_away(premier_league.GW10)
premier_league['GW11'] = home_away(premier_league.GW11)
premier_league['GW12'] = home_away(premier_league.GW12)
premier_league['GW13'] = home_away(premier_league.GW13)
premier_league['GW14'] = home_away(premier_league.GW14)
premier_league['GW15'] = home_away(premier_league.GW15)
premier_league['GW16'] = home_away(premier_league.GW16)

In [21]:
premier_league = premier_league[['Teams', 'Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', 'Points', 'GW7', 'GW8', 'GW9', 'GW10', 'GW11', 'GW12', 'GW13', 'GW14', 'GW15', 'GW16']]

In [22]:
premier_league.index = np.arange(1, len(premier_league) +1)

In [23]:
premier_league

,Teams,Played,Won,Drawn,Lost,GF,GA,GD,Points,GW7,GW8,GW9,GW10,GW11,GW12,GW13,GW14,GW15,GW16
1,Arsenal,6,5,0,1,14,7,7,15,Everton (H),Brentford (A),Tottenham Hotspur (H),Liverpool (H),Leeds United (A),Manchester City (H),Southampton (A),Nottingham Forest (H),Chelsea (A),Wolverhampton Wanderers (A)
2,Manchester City,6,4,2,0,20,6,14,14,Tottenham Hotspur (H),Wolverhampton Wanderers (A),Manchester United (H),Southampton (H),Liverpool (A),Arsenal (A),Brighton & Hove Albion (H),Leicester City (A),Fulham (H),Brentford (H)
3,Tottenham Hotspur,6,4,2,0,12,5,7,14,Manchester City (A),Leicester City (H),Arsenal (A),Brighton & Hove Albion (A),Everton (H),Manchester United (A),Newcastle United (H),AFC Bournemouth (A),Liverpool (H),Leeds United (H)
4,Brighton & Hove Albion,6,4,1,1,11,5,6,13,AFC Bournemouth (A),Crystal Palace (H),Liverpool (A),Tottenham Hotspur (H),Brentford (A),Nottingham Forest (H),Manchester City (A),Chelsea (H),Wolverhampton Wanderers (A),Aston Villa (H)
5,Manchester United,6,4,0,2,8,8,0,12,Crystal Palace (A),Leeds United (H),Manchester City (A),Everton (A),Newcastle United (H),Tottenham Hotspur (H),Chelsea (A),West Ham United (H),Aston Villa (A),Fulham (A)
6,Chelsea,6,3,1,2,8,9,-1,10,Fulham (A),Liverpool (H),Crystal Palace (A),Wolverhampton Wanderers (H),Aston Villa (A),Brentford (A),Manchester United (H),Brighton & Hove Albion (A),Arsenal (H),Newcastle United (A)
7,Liverpool,6,2,3,1,15,6,9,9,Wolverhampton Wanderers (H),Chelsea (A),Brighton & Hove Albion (H),Arsenal (A),Manchester City (H),West Ham United (H),Nottingham Forest (A),Leeds United (H),Tottenham Hotspur (A),Southampton (H)
8,Brentford,6,2,3,1,15,9,6,9,Southampton (A),Arsenal (H),AFC Bournemouth (A),Newcastle United (A),Brighton & Hove Albion (H),Chelsea (H),Aston Villa (A),Wolverhampton Wanderers (H),Nottingham Forest (A),Manchester City (A)
9,Leeds United,6,2,2,2,10,10,0,8,Nottingham Forest (H),Manchester United (A),Aston Villa (H),Crystal Palace (A),Arsenal (H),Leicester City (A),Fulham (H),Liverpool (A),AFC Bournemouth (H),Tottenham Hotspur (A)
10,Fulham,6,2,2,2,9,9,0,8,Chelsea (H),Nottingham Forest (A),Newcastle United (H),West Ham United (A),AFC Bournemouth (H),Aston Villa (H),Leeds United (A),Everton (H),Manchester City (A),Manchester United (H)


In [24]:
data = {'Teams' : ['Arsenal', 'Manchester City', 'Tottenham Hotspur', 'Brighton & Hove Albion', 'Manchester United', 'Chelsea', 'Liverpool', 'Brentford', 'Leeds United', 'Fulham', 'Newcastle United',
    'Southampton', 'AFC Bournemouth', 'Wolverhampton Wanderers', 'Crystal Palace', 'Everton', 'Aston Villa', 'West Ham United', 'Nottingham Forest', 'Leicester City']}

difficulty = pd.DataFrame(data)

In [25]:
def f(x):
    if x < 5:
        return 5
    elif 5 <= x <= 8:
        return 4
    elif 9 <= x <= 13:
        return 3
    elif 14 <= x <= 17:
        return 2
    elif 18 <= x <= 20:
        return 1

In [26]:
difficulty['Position'] = premier_league['Teams'].index

difficulty['Form_Rating'] = difficulty['Position'].apply(lambda x: f(x))

In [27]:
last_season = pd.read_csv("2122 table.csv")

In [28]:
last_season['Team'] = last_season['Team'].replace("Manchester City (C)", "Manchester City")
last_season['Team'] = last_season['Team'].replace("Bournemouth AFC", "AFC Bournemouth")

In [29]:
difficulty = difficulty.merge(last_season, left_on='Teams', right_on='Team')

In [30]:
difficulty = difficulty.drop('Team', axis=1)
difficulty = difficulty.rename(columns={"Pos": "Last_Season_Posn"})

In [31]:
difficulty['Previous_Rating'] = difficulty['Last_Season_Posn'].apply(lambda x: f(x))
difficulty['Overall'] = (difficulty['Form_Rating']) + difficulty['Previous_Rating']

In [32]:
def level(x):
    if x <= 3:
        return 1
    elif 3 <= x <= 4:
        return 2
    elif 5 <= x <= 7:
        return 3
    elif 8 <= x <= 9:
        return 4
    elif x >= 10:
        return 5

In [33]:
difficulty['Level'] = difficulty['Overall'].apply(lambda x: level(x))

In [34]:
difficulty_mapping = dict(zip(difficulty.Teams, difficulty.Level))

In [35]:
def fixture_difficulty(x):
    for oppo, oppo_level in zip(difficulty.Teams, difficulty.Level):
        if x.startswith(oppo):
            return oppo_level

In [36]:
premier_league['Fixture_Difficulty_GW8-GW16'] = (
    premier_league['GW8'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW9'].apply(lambda x: fixture_difficulty(x)) +
    premier_league['GW10'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW11'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW12'].apply(lambda x: fixture_difficulty(x)) +
    premier_league['GW13'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW14'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW15'].apply(lambda x: fixture_difficulty(x))
)    

In [37]:
premier_league['Fixture_Difficulty_GW8-GW12'] = (
    premier_league['GW8'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW9'].apply(lambda x: fixture_difficulty(x)) +
    premier_league['GW10'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW11'].apply(lambda x: fixture_difficulty(x)) + premier_league['GW12'].apply(lambda x: fixture_difficulty(x))
)

In [38]:
premier_league

,Teams,Played,Won,Drawn,Lost,GF,GA,GD,Points,GW7,...,GW9,GW10,GW11,GW12,GW13,GW14,GW15,GW16,Fixture_Difficulty_GW8-GW16,Fixture_Difficulty_GW8-GW12
1,Arsenal,6,5,0,1,14,7,7,15,Everton (H),...,Tottenham Hotspur (H),Liverpool (H),Leeds United (A),Manchester City (H),Southampton (A),Nottingham Forest (H),Chelsea (A),Wolverhampton Wanderers (A),28,20
2,Manchester City,6,4,2,0,20,6,14,14,Tottenham Hotspur (H),...,Manchester United (H),Southampton (H),Liverpool (A),Arsenal (A),Brighton & Hove Albion (H),Leicester City (A),Fulham (H),Brentford (H),27,18
3,Tottenham Hotspur,6,4,2,0,12,5,7,14,Manchester City (A),...,Arsenal (A),Brighton & Hove Albion (A),Everton (H),Manchester United (A),Newcastle United (H),AFC Bournemouth (A),Liverpool (H),Leeds United (H),26,17
4,Brighton & Hove Albion,6,4,1,1,11,5,6,13,AFC Bournemouth (A),...,Liverpool (A),Tottenham Hotspur (H),Brentford (A),Nottingham Forest (H),Manchester City (A),Chelsea (H),Wolverhampton Wanderers (A),Aston Villa (H),28,16
5,Manchester United,6,4,0,2,8,8,0,12,Crystal Palace (A),...,Manchester City (A),Everton (A),Newcastle United (H),Tottenham Hotspur (H),Chelsea (A),West Ham United (H),Aston Villa (A),Fulham (A),27,18
6,Chelsea,6,3,1,2,8,9,-1,10,Fulham (A),...,Crystal Palace (A),Wolverhampton Wanderers (H),Aston Villa (A),Brentford (A),Manchester United (H),Brighton & Hove Albion (A),Arsenal (H),Newcastle United (A),27,15
7,Liverpool,6,2,3,1,15,6,9,9,Wolverhampton Wanderers (H),...,Brighton & Hove Albion (H),Arsenal (A),Manchester City (H),West Ham United (H),Nottingham Forest (A),Leeds United (H),Tottenham Hotspur (A),Southampton (H),29,20
8,Brentford,6,2,3,1,15,9,6,9,Southampton (A),...,AFC Bournemouth (A),Newcastle United (A),Brighton & Hove Albion (H),Chelsea (H),Aston Villa (A),Wolverhampton Wanderers (H),Nottingham Forest (A),Manchester City (A),23,17
9,Leeds United,6,2,2,2,10,10,0,8,Nottingham Forest (H),...,Aston Villa (H),Crystal Palace (A),Arsenal (H),Leicester City (A),Fulham (H),Liverpool (A),AFC Bournemouth (H),Tottenham Hotspur (A),24,16
10,Fulham,6,2,2,2,9,9,0,8,Chelsea (H),...,Newcastle United (H),West Ham United (A),AFC Bournemouth (H),Aston Villa (H),Leeds United (A),Everton (H),Manchester City (A),Manchester United (H),21,11


In [39]:
form_table = premier_league[['Teams', 'GW8', 'GW9', 'GW10', 'GW11', 'GW12', 'Fixture_Difficulty_GW8-GW12']].sort_values('Fixture_Difficulty_GW8-GW12')
form_table.index = np.arange(1, len(form_table) +1)
form_table

,Teams,GW8,GW9,GW10,GW11,GW12,Fixture_Difficulty_GW8-GW12
1,Fulham,Nottingham Forest (A),Newcastle United (H),West Ham United (A),AFC Bournemouth (H),Aston Villa (H),11
2,Aston Villa,Southampton (H),Leeds United (A),Nottingham Forest (A),Chelsea (H),Fulham (A),13
3,Newcastle United,AFC Bournemouth (H),Fulham (A),Brentford (H),Manchester United (A),Everton (H),13
4,West Ham United,Everton (A),Wolverhampton Wanderers (H),Fulham (H),Southampton (A),Liverpool (A),14
5,AFC Bournemouth,Newcastle United (A),Brentford (H),Leicester City (H),Fulham (A),Southampton (H),14
6,Southampton,Aston Villa (A),Everton (H),Manchester City (A),West Ham United (H),AFC Bournemouth (A),14
7,Nottingham Forest,Fulham (H),Leicester City (A),Aston Villa (H),Wolverhampton Wanderers (A),Brighton & Hove Albion (A),14
8,Leicester City,Tottenham Hotspur (A),Nottingham Forest (H),AFC Bournemouth (A),Crystal Palace (H),Leeds United (H),14
9,Chelsea,Liverpool (H),Crystal Palace (A),Wolverhampton Wanderers (H),Aston Villa (A),Brentford (A),15
10,Leeds United,Manchester United (A),Aston Villa (H),Crystal Palace (A),Arsenal (H),Leicester City (A),16


In [40]:
data_players = pd.read_csv('fpl stats.csv', encoding= 'unicode_escape')

In [41]:
data_players.loc[(data_players['Team']=='Brighton'), ['Team']] = 'Brighton & Hove Albion'
data_players.loc[(data_players['Team']=='Man City'), ['Team']] = 'Manchester City'
data_players.loc[(data_players['Team']=='Spurs'), ['Team']] = 'Tottenham Hotspur'
data_players.loc[(data_players['Team']=='Newcastle'), ['Team']] = 'Newcastle United'
data_players.loc[(data_players['Team']=='Leicester'), ['Team']] = 'Leicester City'
data_players.loc[(data_players['Team']=='West Ham'), ['Team']] = 'West Ham United'
data_players.loc[(data_players['Team']=='Wolves'), ['Team']] = 'Wolverhampton Wanderers'
data_players.loc[(data_players['Team']=='Man Utd'), ['Team']] = 'Manchester United'
data_players.loc[(data_players['Team']=="Nott'm Forest"), ['Team']] = 'Nottingham Forest'
data_players.loc[(data_players['Team']=="Bournemouth"), ['Team']] = 'AFC Bournemouth'
data_players.loc[(data_players['Team']=="Leeds"), ['Team']] = 'Leeds United'

In [42]:
data_players['Points'] = pd.to_numeric(data_players['Points'], errors='coerce')

data_players['Cost'] = data_players['Cost'].str.replace("£", "")
data_players['Cost'] = data_players['Cost'].str.replace(".", "")
data_players['Cost'] = pd.to_numeric(data_players['Cost'], errors='coerce')
data_players['Cost'] = data_players['Cost'] / 100

C:\Users\Rob\AppData\Local\Temp/ipykernel_7484/1389223674.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_players['Cost'] = data_players['Cost'].str.replace(".", "")


In [43]:
merge_fixture = form_table[['Teams', 'Fixture_Difficulty_GW8-GW12']]
merge_difficulty = difficulty[['Teams', 'Level']]

In [44]:
data_players = data_players.merge(merge_fixture, left_on='Team', right_on='Teams')
data_players = data_players.merge(merge_difficulty, left_on='Team', right_on='Teams')

In [45]:
data_players = data_players.drop(['Teams_x', 'Teams_y'], axis=1)
data_players = data_players.rename(columns={'Level' : 'Team Level'})

In [70]:
data_players['Form'] = data_players['Points'] / 6
data_players['Form_Value'] = data_players['Form'] /  data_players['Cost']
data_players['Expected_Points'] = data_players['Form'] + 0.5 *(data_players['Team Level'] - (data_players['Fixture_Difficulty_GW8-GW12']/5))
data_players['Expected_Points_Value'] = data_players['Expected_Points'] /  data_players['Cost']

In [74]:
data_players.sort_values('Expected_Points_Value', ascending=False).head(50)

,Player,Team,Position,Points,Cost,Fixture_Difficulty_GW8-GW12,Team Level,Form,Value_Points,Expected_Points,Form_Value,Expected_Points_Value
348,Groß,Brighton & Hove Albion,Midfielder,42.0,6.0,16,4,7.000000,1.166667,7.400000,1.166667,1.233333
349,Mac Allister,Brighton & Hove Albion,Midfielder,39.0,5.6,16,4,6.500000,1.160714,6.900000,1.160714,1.232143
97,Pope,Newcastle United,Goalkeeper,36.0,5.1,13,3,6.000000,1.176471,6.200000,1.176471,1.215686
184,Sá,Wolverhampton Wanderers,Goalkeeper,35.0,5.0,16,3,5.833333,1.166667,5.733333,1.166667,1.146667
335,Sánchez,Brighton & Hove Albion,Goalkeeper,28.0,4.7,16,4,4.666667,0.992908,5.066667,0.992908,1.078014
322,Rashford,Manchester United,Midfielder,40.0,6.5,18,4,6.666667,1.025641,6.866667,1.025641,1.056410
339,Veltman,Brighton & Hove Albion,Defender,26.0,4.6,16,4,4.333333,0.942029,4.733333,0.942029,1.028986
69,Dier,Tottenham Hotspur,Defender,26.0,5.0,17,5,4.333333,0.866667,5.133333,0.866667,1.026667
189,Collins,Wolverhampton Wanderers,Defender,28.0,4.5,16,3,4.666667,1.037037,4.566667,1.037037,1.014815
493,Toney,Brentford,Forward,45.0,7.2,17,3,7.500000,1.041667,7.300000,1.041667,1.013889


In [ ]:
data_players.sort_values('Expected_Points', ascending=False).head(20)